# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [8]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [9]:
!cat teaching.tsv

title	type	url_slug	venue	date	location	teaching_url	description
Basic principles and applications in biological systems	teaching	t1	Instituto de biotecnología, UNAM	2013-08-01	Cuernavaca, Morelos. México		
Fluorescence in the study of proteins. Theoretical course of Introduction to study of proteins	teaching	t2	Instituto de biotecnología, UNAM	2014-03-01	Cuernavaca, Morelos. México		
Fluorescence correlation spectroscopy. Data analysis with simFCS	teaching	t3	Instituto de biotecnología, UNAM	2014-03-01	Cuernavaca, Morelos. México		
Recent fluorescent methodologies to study protein-protein interaction	teaching	t4	Instituto de biotecnología, UNAM	2014-10-01	Cuernavaca, Morelos. México		
Fluorescence resonant energy transfer	teaching	t5	Instituto de biotecnología, UNAM	2015-03-01	Cuernavaca, Morelos. México		
Recent fluorescent methodologies to study protein-protein interaction	teaching	t6	Instituto de biotecnología, UNAM	2016-10-01	Cuernavaca, Morelos. México		
Recent fluorescent method

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [10]:
teaching = pd.read_csv("teaching.tsv", sep="\t", header=0)
teaching

,title,type,url_slug,venue,date,location,teaching_url,description
0,Basic principles and applications in biologica...,teaching,t1,"Instituto de biotecnología, UNAM",2013-08-01,"Cuernavaca, Morelos. México",NaN,NaN
1,Fluorescence in the study of proteins. Theoret...,teaching,t2,"Instituto de biotecnología, UNAM",2014-03-01,"Cuernavaca, Morelos. México",NaN,NaN
2,Fluorescence correlation spectroscopy. Data an...,teaching,t3,"Instituto de biotecnología, UNAM",2014-03-01,"Cuernavaca, Morelos. México",NaN,NaN
3,Recent fluorescent methodologies to study prot...,teaching,t4,"Instituto de biotecnología, UNAM",2014-10-01,"Cuernavaca, Morelos. México",NaN,NaN
4,Fluorescence resonant energy transfer,teaching,t5,"Instituto de biotecnología, UNAM",2015-03-01,"Cuernavaca, Morelos. México",NaN,NaN
5,Recent fluorescent methodologies to study prot...,teaching,t6,"Instituto de biotecnología, UNAM",2016-10-01,"Cuernavaca, Morelos. México",NaN,NaN
6,Recent fluorescent methodologies to study prot...,teaching,t7,"Instituto de biotecnología, UNAM",2017-10-01,"Cuernavaca, Morelos. México",NaN,NaN
7,Recent fluorescent methodologies to study prot...,teaching,t8,"Instituto de biotecnología, UNAM",2018-10-01,"Cuernavaca, Morelos. México",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [11]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [12]:
loc_dict = {}

for row, item in teaching.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: teaching" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Class"\n'
    
    md += "permalink: /teaching/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.teaching_url)) > 3:
        md += "\n[More information here](" + item.teaching_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_teaching/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [13]:
!ls ../_teaching

2013-08-01-t1.md  2014-spring-teaching-1.md  2017-10-01-t7.md
2014-03-01-t2.md  2015-03-01-t5.md	     2018-10-01-t8.md
2014-03-01-t3.md  2015-spring-teaching-2.md
2014-10-01-t4.md  2016-10-01-t6.md


In [14]:
!cat ../_teaching/2013-03-01-tutorial-1.md

cat: ../_teaching/2013-03-01-tutorial-1.md: No such file or directory
